In [1]:
#P13  In-Class 15481075 이재근
#필요한 feature들을 추출하여 테스트 모델을 생성하고 비교하는 과정을 진행하였다. 타이타닉 csv파일을 이용하여 생존과 관련된 예측모델을 만들면서 모델에게 학습시킬 features를 골라 학습이 원할하게 진행될 수 있도록 알맞게 전처리 과정을 진행했다. 데이터 전처리 과정에서 train과 test데이터를 같은 방법으로 한 번에 처리하는 과정에서 두 개의 데이터를 합치는 과정도 진행했다. null값을 처리하는 과정도 간단한 하게 진행하는 것이 신기했고 더 많은 예측 모델을 만들어 비교해보는 과정도 공부해봐야겠다는 생각이 들었습니다.

In [2]:
import findspark

In [3]:
findspark.init('C:\spark-2.4.5-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lgreg').getOrCreate()

In [6]:
data = spark.read.csv('customer_churn.csv',inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [9]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
final_data = output.select('features','Churn')

In [14]:
final_data.head(1)

[Row(features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]), Churn=1)]

In [15]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
lr_churn = LogisticRegression(labelCol='Churn')

In [18]:
fitted_churn_model = lr_churn.fit(train_churn)

In [19]:
training_sum = fitted_churn_model.summary

In [20]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             Churn|         prediction|
+-------+------------------+-------------------+
|  count|               649|                649|
|   mean|0.1694915254237288|0.12788906009244994|
| stddev|0.3754748684760184| 0.3342238290730587|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [21]:
result = fitted_churn_model.transform(test_churn)

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [24]:
result.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [25]:
ACC = my_eval.evaluate(result)

In [26]:
ACC

0.7364336492890995

In [27]:
data2 = spark.read.csv('new_customers.csv',inferSchema=True, header=True)

In [28]:
data2.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          

In [29]:
assembler2 = VectorAssembler(inputCols=['Age',
                                        'Total_Purchase',
                                        'Account_Manager',
                                        'Years',
                                        'Num_Sites'],outputCol='features')

In [30]:
output2 = assembler2.transform(data2)

In [31]:
final_data2 = output2.select('features')

In [33]:
final_data2.head(1)

[Row(features=DenseVector([37.0, 9935.53, 1.0, 7.71, 8.0]))]

In [34]:
fitted_churn_model.transform(final_data2)

DataFrame[features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [35]:
fitted_churn_model.transform(final_data2).show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[37.0,9935.53,1.0...|[2.52199443714297...|[0.92566939975899...|       0.0|
|[23.0,7526.94,1.0...|[-6.3811815560340...|[0.00169025944246...|       1.0|
|[65.0,100.0,1.0,1...|[-4.1712280700687...|[0.01519872884258...|       1.0|
|[32.0,6487.5,0.0,...|[-5.5017391781077...|[0.00406309390987...|       1.0|
|[32.0,13147.71,1....|[1.37254311619143...|[0.79779072152646...|       0.0|
|[22.0,8445.26,1.0...|[-1.5580840423752...|[0.17392174639310...|       1.0|
+--------------------+--------------------+--------------------+----------+

